In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.7.0


In [2]:
!python -V

Python 3.10.18


In [14]:
import pickle
import pandas as pd
import numpy as np

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [10]:
test = pd.read_parquet('/Users/serhatkayikci/Documents/own-projects/mlops-zoomcamp/data/yellow_tripdata_2023-03.parquet')
print(test.columns)


Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee'],
      dtype='object')


In [11]:
df = read_data('/Users/serhatkayikci/Documents/own-projects/mlops-zoomcamp/data/yellow_tripdata_2023-03.parquet')

In [12]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [13]:
print(y_pred)

[16.24590642 26.1347962  11.88426424 ... 11.59533603 13.11317847
 12.89999218]


In [15]:
std_pred = np.std(y_pred)
print(f"Standard deviation of predictions: {std_pred:.2f}")

Standard deviation of predictions: 6.25


In [17]:
year = 2023
month = 3

df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype(str)


In [19]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['prediction'] = y_pred


In [23]:
output_file = f'output/predictions_{year:04d}_{month:02d}.parquet' # Make sure output folder exists under hw4


df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [24]:
import os

file_size_bytes = os.path.getsize(output_file)
file_size_mb = file_size_bytes / (1024 * 1024)

print(f"File size: {file_size_mb:.2f} MB")

File size: 65.46 MB
